In [30]:
from collections import defaultdict as ddict, OrderedDict as odict
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.Chem.Descriptors import MolWt
from sklearn.ensemble import RandomForestRegressor
import sklearn
from rdkit.Chem.rdmolops import GetFormalCharge
import torch
import deepchem as dc
import copy
from sklearn.model_selection import train_test_split
import imp

In [31]:
import sys
sys.path.append('../')
from modules.RNN import double_RNN, RNN
from modules.MPNN import double_MPNN, MPNN
import modules.pretraining as p

---
## Training + testing

### Single input models

In [32]:
s_DMPNN = p.Model(name='DMPNN',
                model=MPNN(MP_depth=3, MP_hidden=256, NN_depth=2, NN_hidden=512, activation='ReLU', 
                                atom_messages=False, dropout=0, readout='sum'),
                lr=0.001,
                batch_size=64,
                data_type='graphs',
                inputs=1)
s_DMPNN_att = p.Model(name='DMPNN with attention',
                      model=MPNN(MP_depth=4, MP_hidden=128, NN_depth=4, NN_hidden=64, activation='ELU', 
                                        atom_messages=False, dropout=0, readout='mean'),
                      lr=0.001,
                      batch_size=64,
                      data_type='graphs',
                      inputs=1)
s_MPNN = p.Model(name='MPNN',
               model=MPNN(MP_depth=3, MP_hidden=256, NN_depth=2, NN_hidden=512, activation='LeakyReLU', 
                          atom_messages=True, dropout=0, readout='sum'),
               lr=0.001,
               batch_size=64,
               data_type='graphs',
               inputs=1)
s_MPNN_att = p.Model(name='MPNN with attention',
                   model=MPNN(MP_depth=2, MP_hidden=64, NN_depth=4, NN_hidden=512, activation='ReLU', 
                                     atom_messages=True, dropout=0, readout='max'),   
                   lr=0.001,
                   batch_size=64,
                   data_type='graphs',
                   inputs=1)
s_RNN = p.Model(name='RNN',
              model=RNN(NN_depth=3, NN_hidden=512, RNN_hidden=512, activation='ReLU', dropout=0.3,
                        features=300, readout='max'),
              lr=0.001,
              batch_size=32,
              data_type='sentences',
              inputs=1)
s_RNN_att = p.Model(name='RNN with attention',
                    model=RNN(NN_depth=1, NN_hidden=1024, RNN_hidden=512, activation='PReLU', dropout=0.1,
                                     features=300, readout='max'),
                    lr=0.001,
                    batch_size=32,
                    data_type='sentences',
                    inputs=1)

#list of all models for testing
s_models = [s_DMPNN, s_DMPNN_att, s_MPNN, s_MPNN_att, s_RNN, s_RNN_att]
s_graph_models = [s_DMPNN, s_DMPNN_att, s_MPNN, s_MPNN_att]
s_sen_models = [s_RNN, s_RNN_att]

### Water pka

In [33]:
data = pd.read_csv('pretrain_data/water_pka.csv')
solute = data['Solute SMILES'].tolist()
pka = data['pKa (avg)'].tolist()
datasets = p.data_maker(solute, pka)

exp_name = "Water pka"
for m in s_models:
    data = datasets[m.data_type]
    print('testing '+m.name+' ...')
    p.fit_no_test(m, exp_name, data)

testing DMPNN ...
testing DMPNN with attention ...
testing MPNN ...
testing MPNN with attention ...
testing RNN ...
testing RNN with attention ...


In [ ]:
max(pka)

### QM9

In [15]:
#data
data = pd.read_csv('pretrain_data/qm9.csv')
smiles = data['smiles'].tolist()
#properties = ['mu','alpha','homo','lumo','gap','r2','zpve','u0','u298','h298','g298','cv']
properties = ['mu','alpha','gap','r2','g298','cv','g298_atom']

In [16]:
#graph models
for prop in properties:
    exp_name = 'QM9_'+prop
    print('---loading '+prop)
    prop_list = data[prop].tolist()
    dataset = p.data_maker_decon(smiles, prop_list, 'graphs')
    prop_list = None
    for m in s_graph_models:
        print('testing '+m.name+' ...')
        p.fit_no_test(m, exp_name, dataset)

---loading mu
testing DMPNN ...


100%|███████████████████████████████████████| 1883/1883 [00:29<00:00, 64.19it/s]


testing DMPNN with attention ...


100%|███████████████████████████████████████| 1883/1883 [00:34<00:00, 54.57it/s]


testing MPNN ...


...

100%|███████████████████████████████████████| 1883/1883 [00:30<00:00, 61.82it/s]


testing MPNN ...


100%|███████████████████████████████████████| 1883/1883 [00:24<00:00, 76.50it/s]


testing MPNN with attention ...


100%|███████████████████████████████████████| 1883/1883 [00:28<00:00, 66.55it/s]


In [ ]:
import importlib 
importlib.reload(p)

#sentence models
for prop in properties:
    exp_name = 'QM9_'+prop
    print('---loading '+prop)
    prop_list = data[prop].tolist()
    dataset = p.data_maker_decon(smiles, prop_list, 'sentences')
    prop_list = None
    for m in s_sen_models:
        print('testing '+m.name+' ...')
        p.fit_no_test(m, exp_name, dataset)

---loading mu


---
### Dual input models

In [24]:
d_DMPNN = p.Model(name='DMPNN',
                model=double_MPNN(MP_depth=3, MP_hidden=256, NN_depth=2, NN_hidden=512, activation='ReLU', 
                                  interaction=None, atom_messages=False, dropout=0, readout='sum'),
                lr=0.001,
                batch_size=64,
                data_type='graphs',
                inputs=2)
d_DMPNN_att = p.Model(name='DMPNN with attention',
                      model=double_MPNN(MP_depth=4, MP_hidden=128, NN_depth=4, NN_hidden=64, activation='ELU', 
                                        atom_messages=False, dropout=0, interaction='tanh', readout='mean'),
                      lr=0.001,
                      batch_size=64,
                      data_type='graphs',
                      inputs=2)
d_MPNN = p.Model(name='MPNN',
                 model=double_MPNN(MP_depth=3, MP_hidden=256, NN_depth=2, NN_hidden=512, activation='LeakyReLU', 
                                   atom_messages=True, dropout=0, interaction=None, readout='sum'),
                 lr=0.001,
                 batch_size=64,
                 data_type='graphs',
                 inputs=2)
d_MPNN_att = p.Model(name='MPNN with attention',
                     model=double_MPNN(MP_depth=2, MP_hidden=64, NN_depth=4, NN_hidden=512, activation='ReLU', 
                                       atom_messages=True, dropout=0, interaction='tanh', readout='max'), 
                     lr=0.001,
                     batch_size=64,
                     data_type='graphs',
                     inputs=2)
d_RNN = p.Model(name='RNN',
                model=double_RNN(NN_depth=3, NN_hidden=512, RNN_hidden=512, activation='ReLU', dropout=0.3,
                                 features=300, interaction=None, readout='max'),
                lr=0.001,
                batch_size=32,
                data_type='sentences',
                inputs=2)
d_RNN_att = p.Model(name='RNN with attention',
                    model=double_RNN(NN_depth=1, NN_hidden=1024, RNN_hidden=512, activation='PReLU', dropout=0.1,
                                     features=300, interaction='exp', readout='max'),
                    lr=0.001,
                    batch_size=32,
                    data_type='sentences',
                    inputs=2)

#list of all models for testing
#d_models = [d_DMPNN, d_DMPNN_att, d_MPNN, d_MPNN_att, d_RNN, d_RNN_att]
d_graph_models = [d_DMPNN, d_DMPNN_att, d_MPNN, d_MPNN_att]
d_sen_models = [d_RNN, d_RNN_att]

---
### Gsolv

In [4]:
# The very last Gsolv value is nan, which breaks the validation loss or training loss at a certain point whenever it arises. To avoid this, I have dropped it here.
data = pd.read_csv('pretrain_data/comp_solv.csv')
data.drop([342158],inplace=True) # This is the last index.

solute = data['mol solvent'].tolist()
solvent = data['mol solute'].tolist()
Gsolv = data['target Gsolv kcal'].tolist()
exp_name = "Gsolv"

In [6]:
graph_datasets = p.data_maker_decon(solute, Gsolv, 'graphs', solvent=solvent)

KeyboardInterrupt: 

In [106]:
import importlib 
importlib.reload(p)

<module 'modules.pretraining' from '../modules/pretraining.py'>

In [17]:
for m in d_graph_models:
    print('testing '+m.name+' ...')
    p.fit_no_test(m, exp_name, graph_datasets)

testing DMPNN ...
tensor([[[ 0.1655,  0.0495, -0.1016,  ..., -0.2221, -0.1312,  0.1659],
         [-0.1367,  0.0673, -0.0521,  ..., -0.1420, -0.4025, -0.2129],
         [ 0.0373, -0.1383, -0.3160,  ..., -0.2490, -0.2950, -0.0161],
         ...,
         [-0.1367,  0.0673, -0.0521,  ..., -0.1420, -0.4025, -0.2129],
         [-0.1367,  0.0673, -0.0521,  ..., -0.1420, -0.4025, -0.2129],
         [ 0.1612,  0.1427, -0.0699,  ..., -0.3203, -0.2415,  0.0655]],

        [[ 0.0723,  0.2225,  0.0227,  ..., -0.1677, -0.2336,  0.1446],
         [ 0.0461,  0.1143, -0.0683,  ..., -0.1665, -0.1754, -0.5153],
         [-0.1259, -0.0966, -0.4584,  ..., -0.0063, -0.4361,  0.1118],
         ...,
         [-0.0845, -0.0193, -0.2386,  ..., -0.2852, -0.3526,  0.0738],
         [ 0.2763, -0.0210, -0.0667,  ...,  0.0091, -0.1528, -0.4056],
         [ 0.3596, -0.1918,  0.2411,  ..., -0.2958, -0.1595, -0.1761]],

        [[ 0.0514,  0.0482, -0.1205,  ...,  0.0075, -0.2370,  0.1635],
         [-0.0090,  0.0072,

AttributeError: 'Tensor' object has no attribute 'get_components'

In [10]:
import importlib 
importlib.reload(p)

<module 'modules.pretraining' from '../modules/pretraining.py'>

In [9]:
sentence_datasets = p.data_maker_decon(solute, Gsolv, 'sentences', solvent=solvent)

In [ ]:
# This will tell you what the index of the nan value is, if there is one.

index = data['target Gsolv kcal'].index[data['target Gsolv kcal'].apply(np.isnan)]
index

In [95]:
# This cell is to check that none of the converted mol2vec sentence tensors do not contain nan values. If they do, then there would be an issue with that row. 
# Fortunately, none of them do have issues.

import tqdm 

problems = []

for i,lst in tqdm.tqdm(enumerate(dataset[0])):
    for j,tensor in enumerate(lst):
        if torch.any(torch.isnan(tensor)):
            problems.append((i,j))

342159it [00:06, 56232.69it/s]


In [11]:
import warnings
warnings.filterwarnings("ignore")

for m in d_sen_models:
    print('testing '+m.name+' ...')
    p.fit_no_test(m, exp_name, sentence_datasets)

testing RNN ...


100%|███████████████████████████████████████| 9624/9624 [02:10<00:00, 73.79it/s]


71.12207455933094


100%|███████████████████████████████████████| 9624/9624 [02:11<00:00, 73.34it/s]


56.66886026505381


...

2.84603847831022


100%|███████████████████████████████████████| 9624/9624 [05:12<00:00, 30.80it/s]


2.9043721532216296


100%|███████████████████████████████████████| 9624/9624 [05:13<00:00, 30.69it/s]


2.917630126554286


In [25]:
from os import listdir
from os.path import isfile, join
trained_models = [f for f in listdir('trained/') if isfile(join('trained/', f))]

def task_func(file):
    if 'Water' in file:
        task = 'Water pKa'
    elif 'Gsolv' in file:
        task = 'Gsolv'
    else:
        task = file[-11:-3]
    return task

model_weights = []
for file in trained_models:
    task = task_func(file)
    if 'RNN_w' in file:
        model_weights.append((d_RNN_att,file,task))
    elif 'DMPNN_w' in file:
        model_weights.append((d_DMPNN_att,file,task))        
    elif 'MPNN_w' in file:
        model_weights.append((d_MPNN_att,file,task))
    elif 'RNN' in file:
        model_weights.append((d_RNN,file,task))
    elif 'DMPNN' in file:
        model_weights.append((d_DMPNN,file,task))
    elif 'MPNN' in file:
        model_weights.append((d_MPNN,file,task))

In [ ]:
def load_exp(model, exp_name, data, train_ids):
    load_model(model, exp_name)
    scaler = pka_scaler(data[1][train_ids])
    model.experiments[exp_name]['scaler'] = scaler
    if model.data_type == 'descriptors':
        scaling_data = data[0][train_ids]
        model.experiments[exp_name]['desc scaling data'] = scaling_data